In [16]:
from pathlib import Path
import json
import re
from tqdm import tqdm
import pandas as pd

settings = json.loads(Path('../../../settings.json').read_text())preprocessed_data_path = Path(settings['preprocessed_data_path'])data_path = Path('.').resolve()data_name = data_path.namesource_name = data_path.parent.namepreprocessed_dir = preprocessed_data_path/source_name/data_namesource_data_dir = preprocessed_dir/'preprocessed'
splits = ['train', 'valid']
tasks = ['번역(한-다국어)']
task_data_dir = preprocessed_dir/'preprocessed_task'
task_data_dir.mkdir(exist_ok=True)
for task in tasks:
    task_path = task_data_dir/task
    task_path.mkdir(exist_ok=True)
    
#### prepare for task preprocess


#### prepare for task preprocess end

#### task preprocess
for split in splits:
    info_dict = {}
    source_data_dir_split = source_data_dir/split
    task_files = [(task_data_dir/task/f'{split}.jsonl').open('w', encoding='utf-8') for task in tasks]
    for source_data in tqdm(list(source_data_dir_split.iterdir()), desc=split):
        source_data = source_data.open()
        new_info_k = {}
        new_info_t = {}
        for line in source_data.readlines():
            line = json.loads(line)
            #### data preprocess
            try:
                if line['k_column'] != '관광타입':
                    new_info_k[line['k_column']] = line['k_context']
                    new_info_t[line['t_column']] = line['t_context']
                else:
                    if new_info_k:
                        name_key = '시설명' if '시설명' in new_info_k else '관광지명'
                        if name_key in new_info_k:
                            if new_info_k[name_key] not in info_dict:
                                info_dict[new_info_k[name_key]] = {'kr':new_info_k, line['language']:new_info_t}
                            else:
                                info_dict[new_info_k[name_key]][line['language']] = new_info_t
                        
                    new_info_k = {line['k_column']:line['k_context']}
                    new_info_t = {line['t_column']:line['t_context']}
            except:
                print(line)
            #### data preprocess end 
                
    for name, info in info_dict.items():
        #### 번역(한-영)
        data = {'input': None, 'output': {}}
        ## preprocess data from line
        for lang, lang_info in info.items():
            df = pd.DataFrame([lang_info])
            text = df.to_markdown(index=False)
            if lang == 'kr':
                data['input'] = text
            else:
                data['output'][lang] = text
        
        
        ## preprocess data from line end
        task_files[0].write(json.dumps(data, ensure_ascii=False)+'\n')
        # print(json.dumps(data, indent=4, ensure_ascii=False))
        #### 번역(한-영) end


            # break
        # break
    # break

    for path in task_files:
        path.close()      
        
#### task preprocess end      

train:  21%|██▏       | 6/28 [00:04<00:20,  1.06it/s]

1


valid: 100%|██████████| 28/28 [00:04<00:00,  6.23it/s]
